In [3]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import os
import dill as pickle
from itertools import combinations
import random
import tensorflow as tf

from keras.models import Model, Input, Sequential, load_model
from keras.applications.vgg16 import VGG16
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten, MaxPooling2D, ZeroPadding2D, Dropout
from keras.optimizers import SGD,Adam
import keras.metrics
from keras.utils import plot_model

from src.data import process_audio
from src.data import display_audio
from src.data import process_audio_siam
from src.models import one_shot_learning
from src.data import process_audio_VGG16_features
from src.models import one_shot_learning_VGG16



/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/greenapple/anacond

In [4]:
%load_ext autoreload
%autoreload 2

# Extract embeddings with VGG16 

## Trying it out

In [7]:
path = '/Users/greenapple/project5/data/raw/LibriSpeech/train-clean-360'
speaker_start = 0
speaker_stop = 10

data_10 = process_audio_VGG16_features.ten_audio_to_VGG16_features(path, speaker_start, speaker_stop)

Done with speaker 1, have 10 audio files
Done with speaker 2, have 10 audio files
Done with speaker 3, have 10 audio files
Done with speaker 4, have 10 audio files
Done with speaker 5, have 10 audio files
Done with speaker 6, have 10 audio files
Done with speaker 7, have 10 audio files
Done with speaker 8, have 10 audio files
Done with speaker 9, have 10 audio files
Done with speaker 10, have 10 audio files
Instructions for updating:
Colocations handled automatically by placer.


In [8]:
data_10.shape

(100, 4)

In [20]:
pickle.dump(data_10, open('/Users/greenapple/project5/data/processed/data_10.pkl', 'wb'))

In [22]:
X_10, y_10, siamese_data_10 = process_audio_VGG16_features.dataframe_to_siamese_features(data_10)

In [23]:
pickle.dump(X_10, open('/Users/greenapple/project5/data/processed/X_10.pkl', 'wb'))

In [24]:
pickle.dump(y_10, open('/Users/greenapple/project5/data/processed/y_10.pkl', 'wb'))

In [25]:
pickle.dump(siamese_data_10, open('/Users/greenapple/project5/data/processed/siamese_data_10.pkl', 'wb'))

In [ ]:
'''
Saving VGG16 embeddings in a dataframe works well. 
Would need to reshape the dataframe to a tensor before modeling. 
Saving the data as a tensor is expensive.
The shape of the embeddings is (7, 7, 520).

'''

## Extract embeddings for 900 speakers from the training set.

In [37]:
# Extract embeddings and organise them in a dataframe.
path = '/Users/greenapple/project5/data/raw/LibriSpeech/train-clean-360'
speaker_start = 0
speaker_stop = 900

data_900 = process_audio_VGG16_features.ten_audio_to_VGG16_features(path, speaker_start, speaker_stop)

Done with speaker 1, have 10 audio files
Done with speaker 2, have 10 audio files
Done with speaker 3, have 10 audio files
Done with speaker 4, have 10 audio files
Done with speaker 5, have 10 audio files
Done with speaker 6, have 10 audio files
Done with speaker 7, have 10 audio files
Done with speaker 8, have 10 audio files
Done with speaker 9, have 10 audio files
Done with speaker 10, have 10 audio files
Done with speaker 11, have 10 audio files
Done with speaker 12, have 10 audio files
Done with speaker 13, have 10 audio files
Done with speaker 14, have 10 audio files
Done with speaker 15, have 10 audio files
Done with speaker 16, have 10 audio files
Done with speaker 17, have 10 audio files
Done with speaker 18, have 10 audio files
Done with speaker 19, have 10 audio files
Done with speaker 20, have 10 audio files
Done with speaker 21, have 10 audio files
Done with speaker 22, have 10 audio files
Done with speaker 23, have 10 audio files
Done with speaker 24, have 10 audio files
D

Done with speaker 214, have 10 audio files
Done with speaker 215, have 10 audio files
Done with speaker 216, have 10 audio files
Done with speaker 217, have 10 audio files
Done with speaker 218, have 10 audio files
Done with speaker 219, have 10 audio files
Done with speaker 220, have 10 audio files
Done with speaker 221, have 10 audio files
Done with speaker 222, have 10 audio files
Done with speaker 223, have 10 audio files
Done with speaker 224, have 10 audio files
Done with speaker 225, have 10 audio files
Done with speaker 226, have 10 audio files
Done with speaker 227, have 10 audio files
Done with speaker 228, have 10 audio files
Done with speaker 229, have 10 audio files
Done with speaker 230, have 10 audio files
Done with speaker 231, have 10 audio files
Done with speaker 232, have 10 audio files
Done with speaker 233, have 10 audio files
Done with speaker 234, have 10 audio files
Done with speaker 235, have 10 audio files
Done with speaker 236, have 10 audio files
Done with s

Done with speaker 430, have 10 audio files
Done with speaker 431, have 10 audio files
Done with speaker 432, have 10 audio files
Done with speaker 433, have 10 audio files
Done with speaker 434, have 10 audio files
Done with speaker 435, have 10 audio files
Done with speaker 436, have 10 audio files
Done with speaker 437, have 10 audio files
Done with speaker 438, have 10 audio files
Done with speaker 439, have 10 audio files
Done with speaker 440, have 10 audio files
Done with speaker 441, have 10 audio files
Done with speaker 442, have 10 audio files
Done with speaker 443, have 10 audio files
Done with speaker 444, have 10 audio files
Done with speaker 445, have 10 audio files
Done with speaker 446, have 10 audio files
Done with speaker 447, have 10 audio files
Done with speaker 448, have 10 audio files
Done with speaker 449, have 10 audio files
Done with speaker 450, have 10 audio files
Done with speaker 451, have 10 audio files
Done with speaker 452, have 10 audio files
Done with s

Done with speaker 636, have 10 audio files
Done with speaker 637, have 10 audio files
Done with speaker 638, have 10 audio files
Done with speaker 639, have 10 audio files
Done with speaker 640, have 10 audio files
Done with speaker 641, have 10 audio files
Done with speaker 642, have 10 audio files
Done with speaker 643, have 10 audio files
Done with speaker 644, have 10 audio files
Done with speaker 645, have 10 audio files
Done with speaker 646, have 10 audio files
Done with speaker 647, have 10 audio files
Done with speaker 648, have 10 audio files
Done with speaker 649, have 10 audio files
Done with speaker 650, have 10 audio files
Done with speaker 651, have 10 audio files
Done with speaker 652, have 10 audio files
Done with speaker 653, have 10 audio files
Done with speaker 654, have 10 audio files
Done with speaker 655, have 10 audio files
Done with speaker 656, have 10 audio files
Done with speaker 657, have 10 audio files
Done with speaker 658, have 10 audio files
Done with s

Done with speaker 862, have 10 audio files
Done with speaker 863, have 10 audio files
Done with speaker 864, have 10 audio files
Done with speaker 865, have 10 audio files
Done with speaker 866, have 10 audio files
Done with speaker 867, have 10 audio files
Done with speaker 868, have 10 audio files
Done with speaker 869, have 10 audio files
Done with speaker 870, have 10 audio files
Done with speaker 871, have 10 audio files
Done with speaker 872, have 10 audio files
Done with speaker 873, have 10 audio files
Done with speaker 874, have 10 audio files
Done with speaker 875, have 10 audio files
Done with speaker 876, have 10 audio files
Done with speaker 877, have 10 audio files
Done with speaker 878, have 10 audio files
Done with speaker 879, have 10 audio files
Done with speaker 880, have 10 audio files
Done with speaker 881, have 10 audio files
Done with speaker 882, have 10 audio files
Done with speaker 883, have 10 audio files
Done with speaker 884, have 10 audio files
Done with s

In [38]:
# Save dataframe:
pickle.dump(data_900, open('/Users/greenapple/project5/data/processed/data_900.pkl', 'wb'))

In [6]:
# Load dataframe:
data_900 = pickle.load(open('/Users/greenapple/project5/data/processed/data_900.pkl', 'rb'))

In [1]:
# Reshape for the siamese model before modeling:
X_900, y_900, siamese_data_900 = process_audio_VGG16_features.dataframe_to_siamese_features(data_900)

NameError: name 'process_audio_VGG16_features' is not defined

## Extract embeddings for 40 speakers from the test set.

In [5]:
# Extract embeddings and organize them in a dataframe:
test_path = '/Users/greenapple/project5/data/raw/LibriSpeech/train-clean-360'
test_speaker_start = 0
test_speaker_stop = 40

test_data_40 = process_audio_VGG16_features.ten_audio_to_VGG16_features(
    test_path, 
    test_speaker_start,
    test_speaker_stop)

Done with speaker 1, have 10 audio files
Done with speaker 2, have 10 audio files
Done with speaker 3, have 10 audio files
Done with speaker 4, have 10 audio files
Done with speaker 5, have 10 audio files
Done with speaker 6, have 10 audio files
Done with speaker 7, have 10 audio files
Done with speaker 8, have 10 audio files
Done with speaker 9, have 10 audio files
Done with speaker 10, have 10 audio files
Done with speaker 11, have 10 audio files
Done with speaker 12, have 10 audio files
Done with speaker 13, have 10 audio files
Done with speaker 14, have 10 audio files
Done with speaker 15, have 10 audio files
Done with speaker 16, have 10 audio files
Done with speaker 17, have 10 audio files
Done with speaker 18, have 10 audio files
Done with speaker 19, have 10 audio files
Done with speaker 20, have 10 audio files
Done with speaker 21, have 10 audio files
Done with speaker 22, have 10 audio files
Done with speaker 23, have 10 audio files
Done with speaker 24, have 10 audio files
D

In [6]:
# Save test set dataframe:
pickle.dump(test_data_40, open(
    '/Users/greenapple/project5/data/processed/test_data_40.pkl', 'wb'
))

In [7]:
test_data_40.shape

(400, 4)